In [4]:
# import libraries
# %pip install -U python-dotenv langchain langchain_community langchain_mongodb nltk langchain-ollama langchain-pinecone pinecone-notebooks einops langgraph langchain_huggingface

assuming you have a JSON copy of the data   
Data aquisition:   
import the json data

In [1]:
# read the json file
import json
from pathlib import Path
from dotenv import load_dotenv
import os
load_dotenv()
# load a json file as data
data  = json.loads(Path("../data/data.json").read_text())

Data ingestion:   
since it is better to supply the full context of structured and unstructured data fetched in the future,   
we will store the raw text in mongodb first   
and apply preprocessing later when we need semantically/meaning accurate search results (important for vectorization)

In [2]:
# Connect to your MongoDB Atlas(Cloud) cluster
from pymongo import MongoClient
client = MongoClient(os.environ["MONGODB_URI"])
db = client[os.environ["MONGODB_DB"]]

In [3]:
tables = [
    {"name":"premium_plans", "field_containing_data":"all"},
    {"name":"critical_illnesses", "field_containing_data":"illnesses"},
    {"name":"schedule_of_compensation","field_containing_data":"all"},
    {"name":"total_investment_estimations", "field_containing_data":"all"},
    {"name":"premium_allocations","field_containing_data":"all"},
    {"name":"claims","field_containing_data":"claim"},
    {"name":"funds","field_containing_data":"fund_performance"},
    ]
for table in tables:
    print(data[table['name']])
    table['table'] = data[table['name']]
    

[{'currency': 'RM', 'premium_per_month': 30, 'coverage': {'critical_illnesses': 20000, 'death_natural': 20000, 'accidental_death': 40000, 'TPD_illness': 20000, 'TPD_accident': 40000, 'hospitalisation_income_per_day': 20, 'funeral_expenses': 5000}}, {'currency': 'RM', 'premium_per_month': 45, 'coverage': {'critical_illnesses': 30000, 'death_natural': 30000, 'accidental_death': 60000, 'TPD_illness': 30000, 'TPD_accident': 60000, 'hospitalisation_income_per_day': 30, 'funeral_expenses': 5000}}]
{'disclaimer': 'Upon payment of claim for Angioplasty and other invasive treatments for coronary artery disease, the sum assured will be reduced by the quantum of the payment for Angioplasty and other invasive treatments for coronary artery disease. However, the premium shall remain unchanged. This benefit is subject to a limit of RM1,000,000 under the Policy and all other non-credit-related group policies (including supplementary contract and endorsement, if any) issued by the Company by any name 

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama as Ollama
from langchain_core.output_parsers import StrOutputParser as parser

# pipe = HuggingFacePipeline.from_model_id(model_id="meta-llama/Llama-3.2-3B",task = "text-generation")

def tables_summarize(tables):
    prompt_text = """You are an assistant tasked with summarizing tables and text. \
                    Give a concise summary of the table or text as sentences. \
                    Do not include text like "here is a summary of the table" in your responses \
                    Table or text chunk: {element} """

    prompt = ChatPromptTemplate.from_template(prompt_text)
    model = Ollama(temperature=0, model="llama3.2")
    summarize_chain = {"element": lambda x: x} | prompt | model | parser()
    table_summaries = summarize_chain.batch(tables, {"max_concurrency": 5})
    
        
    #text_summaries =  summarize_chain.batch(data_category[0], {"max_concurrency": 5})# no need to summarize

    return table_summaries
# table_summaries = tables_summarize(tables)



In [9]:
summaries = {}
for index, value in enumerate(table_summaries):
    summaries[tables[index]["name"]] = value

for key, value in summaries.items():
    if isinstance(data[key],list) & (len(data[key])>1):
        collection = {"table":data[key]}
    else:
        collection = data[key]
    collection["summary"] = value
    data[key] = collection
    print(data[key])



{'table': [{'currency': 'RM', 'premium_per_month': 30, 'coverage': {'critical_illnesses': 20000, 'death_natural': 20000, 'accidental_death': 40000, 'TPD_illness': 20000, 'TPD_accident': 40000, 'hospitalisation_income_per_day': 20, 'funeral_expenses': 5000}}, {'currency': 'RM', 'premium_per_month': 45, 'coverage': {'critical_illnesses': 30000, 'death_natural': 30000, 'accidental_death': 60000, 'TPD_illness': 30000, 'TPD_accident': 60000, 'hospitalisation_income_per_day': 30, 'funeral_expenses': 5000}}], 'summary': 'The table contains information on premium plans for a life insurance policy. The two plans have different monthly premiums and coverage amounts for various benefits such as critical illnesses, death due to natural causes, accidental deaths, and hospitalization income per day. The plans also include funeral expenses coverage.'}
{'disclaimer': 'Upon payment of claim for Angioplasty and other invasive treatments for coronary artery disease, the sum assured will be reduced by the

In [5]:
#specify outer level json objects as mongodb collection names, then based on the specified names, create new collections and insert corresponding data
#specify all the collections
collection_names = []
for key, value in data.items():
    collection_names.append(key)
    # if type(value) != str:
    #     value = json.dumps(value)
    # print(key+":    "+value)
    
#fetch existing collections
ignored_and_existing_collections = db.list_collection_names()

#specify the collections to ignore
if "questions_and_answers2" not in ignored_and_existing_collections:
        ignored_and_existing_collections.append("questions_and_answers2")
if "exclusions2" not in ignored_and_existing_collections:
        ignored_and_existing_collections.append("exclusions2")

for collection_name in collection_names:
    if collection_name in ignored_and_existing_collections:
       continue
    collection = db[collection_name]
    if isinstance(data[collection_name], list):
        print(f"""
inserting many records of {data[collection_name]} into {collection_name} collection
""")
        collection.insert_many(data[collection_name])
    else:
        print(f"""
inserting one record of {data[collection_name]} into {collection_name} collection
""")
        collection.insert_one(data[collection_name])

# separately insert the vector targets
collection = db["vector_targets"]
vector_target_records = list(collection.find())
if len(vector_target_records) == 0:
        print(f"""
inserting one record of {data["vector_targets"]} into {collection_name} collection
""")
        collection.insert_many(data["vector_targets"])        
else:
    print("vector_targets already exists in the database, skipping insertion")

print("insertion complete")

ignored_and_existing_collections = None
del ignored_and_existing_collections
collection_names = None
del collection_names
vector_target_records = None
del vector_target_records
data = None
del data


vector_targets already exists in the database, skipping insertion
insertion complete


further preprocessing:   
lowercasing   
special character removal

stop words removal

lemmatization

In [6]:
#use nltk to lowercase, remove special characters, tokenize, remove stopwords, and lemmatize the values in the json data
#fetch all data cleaning resources
import nltk
import re
try:
    nltk.data.find('corpora/wordnet.zip')
except:
    print("sam ting wong")
    nltk.download('wordnet')
    
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

#initialize the lemmatizer
lemmatizer = WordNetLemmatizer()
#initialize the stopwords
stop_words = set(stopwords.words('english'))
#allow specific stopwords
stop_words.remove("both")


def preprocess_text(text):
    pattern = re.compile(r"[;@#&*!()\[\]]")
    def get_wordnet_pos(tag):
        match tag[0]:
            case 'J':
                return wordnet.ADJ
            case 'V':
                return wordnet.VERB
            case 'R':
                return wordnet.ADV
            case _:
                return wordnet.NOUN
    # Tokenize
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    # Remove stop words and lemmatize
    processed_tokens = [
        lemmatizer.lemmatize(word.lower(), get_wordnet_pos(tag)) 
        for word, tag in pos_tags 
        if word.lower() not in stop_words and not pattern.match(word)
    ]
    # Join tokens back to a string
    return ' '.join(processed_tokens)

#setup a function to recursively preprocess the json data
def traverse_json(data, json_text_action, target_key=None):
    if isinstance(data, dict):
        for key, value in data.items():
            if isinstance(value, str) and (target_key is None or key == target_key):
                # Preprocess the string value
                data[key] = json_text_action(value)
            elif isinstance(value, dict) or isinstance(value, list):
                # Recursively preprocess the dictionary or list
                traverse_json(value, json_text_action)
    elif isinstance(data, list):
        for i, element in enumerate(data):
            if isinstance(element, str):
                # Preprocess the string in the list and save it in the original position
                data[i] = json_text_action(element)
            elif isinstance(element, dict) or isinstance(element, list):
                # Recursively preprocess the dictionary or list
                traverse_json(element, json_text_action)
    return data

# preprocess_text("i need information on performing funds. how much my premium is allocated? the allocation rate, the pricing plan, how much will i be paying per premium payment, what is the coverage performance, what are the fundings, testing the word post-apocalyptic, 80 90% 20$")

In [7]:
# Connect to your pinecone vector database and specify the working index
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(os.environ["PINECONE_API_KEY"])
index = None
if os.environ["PINECONE_INDEX_NAME"] not in [index.name for index in pc.list_indexes()]:
    index = pc.create_index(name = os.environ["PINECONE_INDEX_NAME"],
                            dimension = 768, 
                            metric = "cosine",
                            spec = ServerlessSpec(cloud = "aws", region = "us-east-1") 
                            )
else:
    index = pc.Index(os.environ["PINECONE_INDEX_NAME"])

/Volumes/KINGSTON SNV2S1000G Media/prototypes/python/chatbot/.venv/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [8]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 51}},
 'total_vector_count': 51}

data embedding//vectorization

target descriptive values and generate embeddings for semantic search

In [9]:
# Load the embedding model (https://huggingface.co/nomic-ai/nomic-embed-text-v1.5")
# %pip install sentence-transformers scipy
# from sentence_transformers import SentenceTransformer
from langchain_ollama import OllamaEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings(model_name="nomic-ai/nomic-embed-text-v1.5", model_kwargs={"trust_remote_code":True})
# global_embedding_model = SentenceTransformer(global_embedding_model = SentenceTransformer("nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True))
global_embedding_model = OllamaEmbeddings(model = 'nomic-embed-text')

# number_of_dimensions = 768 #nomic-embed-text-v1.5 has 768 dimensions
vector_store = PineconeVectorStore(index=index, embedding=global_embedding_model)

<All keys matched successfully>


In [10]:
#specify the target fields to embed
from copy import deepcopy
from langchain_core.documents import Document

def vectorize(texts):
    "vectorize the provided texts using a globally declared model."    
    vectors = global_embedding_model.encode(texts)
    return vectors

ignore_collections = ["vector_targets", "premium_plans","important_notice","total_investment_estimations","premium_allocations","test2", "test","questions_and_answers2","exclusions2"]

def vectorize_and_index(mongo_database=db, vector_index=index, ignore_collections=ignore_collections, debug=False):
    """vectorize the target fields and store the embeddings in the pinecone index.
    
current implmentation does not support converting mongo collection name to pinecone namespaces."""
    if mongo_database == None:
        raise ValueError("mongo_database is required, the mongo database should also have a vector_targets collection specifying the target fields to vectorize")
    if not vector_index:
        raise ValueError("vector_index is required")
    documents = []
    for collection_name in mongo_database.list_collection_names():
        if collection_name in ignore_collections:
            continue                                            #skip collections that should not to be vectorized
        if debug:
            print(f"\nIn {collection_name}")
        results = list(mongo_database[collection_name].find())
        targets = list(mongo_database.vector_targets.find({"name": collection_name}))
        for record in results:
            if targets:
                for target in targets:                              #iterate through the target fields from the same collection
                    old_data = deepcopy(record[target['field']])              #store original info
                    traverse_json(record, preprocess_text, target['field'])
                    cleaned_data = record[target['field']]                     #clean the data
                    # vector = vectorize(record[target['field']])                #store the vectorized embeddings
                    if debug:
                        print(f"original: {old_data}\ncleaned : {cleaned_data}")
                    if isinstance(cleaned_data, list):#check if the vectorized embeddings are of the correct length
                        #if the vectorized embeddings length not correct, possibility of other datatypes, iterate through the iterable
                        for j , sub_vector in enumerate(cleaned_data):
                            if debug:
                                print(f"vectorizing item {j+1} : {cleaned_data[j]}")
                            documents.append(Document(
                                            page_content = cleaned_data[j]
                                            ,metadata = {"original":old_data[j]}
                                            ,id = str(record['_id'])+"-"+collection_name+"-"+target['field']+"-"+str(j) 
                            ))
                    else:   
                        print("vectorizing string")
                        documents.append(Document(
                                        page_content=cleaned_data
                                        ,metadata={"original":old_data}
                                        ,id = str(record['_id'])+"-"+collection_name+"-"+target['field'] 
                        ))
                        pass                                        
                        
            else:
                if debug:
                    print("no matching target, skipping vectorization")
                continue
            
    vector_store.add_documents(documents=documents)

# vectorize_and_index(debug=True)


In [11]:
#using local(ollama) base model as chatbot model
intent_classifier = Ollama(model='llama3.2',#:1B
                           temperature=0,
                           num_predict=4,
                           request_timeout=10,
                           verbose=False )
chatbot_model = Ollama(model='llama3.2', request_timeout=60)

In [14]:
#setup the semantic search function
def semantic_search(question, vector_store=vector_store, index=index, top_k=6, verbose=False):
    """vectorizes the question and queries the pinecone index for the top 3 closest matches.

current implementation does not support querying multiple namespaces or using mongoDB indexes.

Args:
    question (str): _description_
    debug (bool, optional): _description_. Defaults to False.
    embedding (SentenceTransformer, optional): _description_. Defaults to global_embedding_model.
    index (Pinecone.Index, optional): _description_. Defaults to index.

Returns:
    list(QueryResponse): the top closest query results from the pinecone index
    """
    if not index:
        print("Index not found, please specify your pinecone or mongo search index")
        return

    if verbose:
        print("Encoding question...")
        
    matches = []
    spaces = index.describe_index_stats()['namespaces']
    for key, value in spaces.items():
        res = vector_store.similarity_search(
            query=question,
            k=top_k
        #     namespace=key,
        )
        matches.append(res)
    return matches

# Example usage
response = semantic_search("who manages this product")
response

[[Document(id='67286e50987ce18efd0c3ee4-questions_and_answers-answer-2', metadata={'original': 'Fund Management Charge is 0.50% per annum. Note: The fees and charges levied may change from time to time.'}, page_content='fund management charge 0.50 % per annum . note : fee charge levy may change time time .'),
  Document(id='67286e50987ce18efd0c3ee1-questions_and_answers-answer', metadata={'original': 'Yes. The Assured Member/spouse/children will be required to reapply by completing a standard Group Proposal form, subject to approval by the Company and up to the maximum benefit allowed'}, page_content='yes . assured member/spouse/children require reapply complete standard group proposal form , subject approval company maximum benefit allow'),
  Document(id='67286e50987ce18efd0c3ec7-advantages-advantage', metadata={'original': 'Automatic premium remittance via credit card / bank deduction / JomPay ensures continuous protection.'}, page_content='automatic premium remittance via credit car

In [15]:

#setup a function to receive semantic search results and return the collection name and ids
def get_collection_matches(response : list, verbose=False) -> list:
    """based on the response from a pinecone semantic search, extract and consolidate the collection name and ids of the matching documents.

    Args:
        response (list): a list of QueryResponse objects from a pinecone semantic search
        verbose (bool, optional): flag to use debug mode. Defaults to False.

    Returns:
        list: list of mongodb documents
    """
    def extract_info(match: str) -> dict:
        data_from_id = None
        try:
            data_from_id = match['id'].split("-")
        except TypeError as e: #probably a document object
            data_from_id = match.id.split("-")
        match_id = data_from_id[0]
        collection_name = data_from_id[1]
        return {'collection':collection_name, 'id':match_id}
    
    if verbose:
        print("Getting collection matches...")
    document_metadata = []
    for namespaces_or_documents in response:
        is_using_default_namespace = False
        if isinstance(namespaces_or_documents, list):
            if verbose:
                print("no pinecone namespace found, checking id or additional metadata")
            for metadata in namespaces_or_documents:
                document = extract_info(metadata)
                if document in document_metadata:
                    if verbose:
                        print(f"Duplicate id {document['id']} for {document['collection']} found in fetch list, ignoring")
                    continue
                document_metadata.append(document)
        else: #probably a dictionary
            document = extract_info(namespaces_or_documents)
            if document in document_metadata:
                if verbose:
                    print(f"Duplicate id {document['id']} for {document['collection']} found in fetch list, ignoring")
                continue
            ids = []
            for match in collection:
                if verbose:
                    print(match)
                data_from_id = match['id'].split("-")
                match_id = data_from_id[0]
                if is_using_default_namespace:
                    collection = data_from_id[1]
                #filter out duplicate ids
                if match_id not in ids:
                    ids.append(match_id)
                else:
                    if verbose:
                        print(f"Duplicate id {match_id} for {collection} found in fetch list, ignoring")
                    continue
                if is_using_default_namespace:
                    document_metadata.append({'collection':collection, 'id':match_id})
            document_metadata.append({'collection':collection, 'ids':ids})
    return document_metadata

document_metadata = get_collection_matches(response, verbose=True)
# print(document_metadata)

Getting collection matches...
no pinecone namespace found, checking id or additional metadata


In [ ]:
#find the mongo documents based on their collection and ids
from bson.objectid import ObjectId as oid 
def find_documents(collection_matches : list, verbose=False, database=db) -> list:
    """use the collection matches to find the corresponding documents in the specified mongo database.

    Args:
        collection_matches (list): list of document metadata containing collection names and ids
        verbose (bool, optional): flag to turn on debug mode. Defaults to False.
        database (pymongo.synchronous.database.Database, optional): the mongodb database to use. Defaults to the db variable.

    Returns:
        list: list of mongodb documents
    """
    print(f"Finding documents using {len(collection_matches)} references") if verbose else None
    documents = []
    for collection_match in collection_matches:
        collection = database[collection_match['collection']]
        if 'ids' in collection_match:
            for id in collection_match['ids']:
                documents.append(collection.find_one({"_id": oid(id)}))
        else:
            documents.append(collection.find_one({"_id": oid(collection_match['id'])}))
    return documents

documents = find_documents(document_metadata)
documents

[{'_id': ObjectId('67286e50987ce18efd0c3ee4'),
  'question': 'What are the current fees and charges?',
  'answer': ['Insurance charges are Applicable to the sum assured, and vary depending on the average age profile and claim experience of the scheme.',
   'Monthly Policy Fee is RM5.00',
   'Fund Management Charge is 0.50% per annum. Note: The fees and charges levied may change from time to time.']},
 {'_id': ObjectId('67286e50987ce18efd0c3ee1'),
  'question': 'Can the Assured Member/spouse/children apply to contribute more',
  'answer': 'Yes. The Assured Member/spouse/children will be required to reapply by completing a standard Group Proposal form, subject to approval by the Company and up to the maximum benefit allowed'},
 {'_id': ObjectId('67286e50987ce18efd0c3ec7'),
  'advantage': 'Automatic premium remittance via credit card / bank deduction / JomPay ensures continuous protection.'},
 {'_id': ObjectId('67286e50987ce18efd0c3ecc'),
  'demographic': 'general',
  'description': 'Assu

the previous 3 functions only allow access to unstructured data, but ignores structured data like tables   
add a component for fetching structured data

In [17]:
#setup the chatbot model for tool use
from langchain_core.tools import tool
import re

@tool
def fetch_metrics(question: str) -> list:
    """Retrieves only JSON of table data for either premium plans, total investment estimations, premium allocation or fund performance based on the user query."""
    cleaned_question = preprocess_text(question)
    documents = []
    has_fetched_preimum_plans = False
    has_fetched_total_investment_estimation = False
    has_fetched_fund = False
    has_fetched_premium_allocation = False
    for keyword in re.findall(r'\b\w+\b', cleaned_question):
        match keyword:
            case "premium" | "plan" | "coverage" | "price" | "pricing":
                if not has_fetched_preimum_plans:
                    print("fetching premium plans")
                    
                    documents.append(list(db["premium_plans"].find()))
                    has_fetched_preimum_plans = True
            case "investment" | "value" | "allocation":
                if not has_fetched_total_investment_estimation:
                    print("fetching investment plans")
                    
                    documents.append(list(db["total_investment_estimations"].find()))
                    has_fetched_total_investment_estimation = True
            case "performance" | "perform" | "fund":
                if not has_fetched_fund:
                    print("fetching fund performance")
                    
                    documents.append(list(db["funds"].find()))
                    has_fetched_fund = True
            case "allocation":
                if not has_fetched_premium_allocation:
                    print("fetching premium allocation")
                    
                    documents.append(list(db["premium_allocations"].find()))
                    has_fetched_premium_allocation = True
            case _:
                continue

    return documents
# @tool
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def get_context(question: str, verbose: bool = False) -> list:
    """Retrieves text-based information for an insurance product only based on the user query.
does not answer quwstions about the chat agent"""
    print(question)
    matches = semantic_search(question, verbose=verbose)
    document_data = get_collection_matches(matches, verbose=verbose)
    context = find_documents(document_data, verbose=verbose)
    # if verbose:
    #     for message in messages:
    #         print(message)
    template = ChatPromptTemplate.from_template(f"""fullfill the query with the provided information
Query      :{{question}}
Information:{{context}}""")

    # RAG pipeline
    chain = {
        "context": lambda x: context , "question": RunnablePassthrough()
        } | template | chatbot_model | StrOutputParser()
    return chain.stream(question)

# tools = [fetch_metrics,
#         get_context
#         ]
# toolPicker = Ollama(model='llama3.2').bind_tools(tools)
# for group in fetch_metrics("i need information on performing funds, how much my premium is allocated, the allocation rate, the pricing plan, how much will i be paying per premium payment, what is the coverage performance, what are the fundings"):
#     for document in group:
#         print(document)

# fetch_metrics("i need information on performing funds. how much my premium is allocated? the allocation rate, the pricing plan, how much will i be paying per premium payment, what is the coverage performance, what are the fundings, testing the word post-apocalyptic, 80 90% 20$")

In [20]:
#setup the RAG workflow
from langchain_core.prompts import ChatPromptTemplate
from pydantic import ValidationError

def RAG(query : str, verbose : bool | None = False):
    if query in ["exit", "quit", "bye", "end", "stop", ""]:
        return
    # if verbose:
    #     tool_query=query+" (Turn on debug mode)"
    # else:
    #     tool_query=query
    # messages = [query]
    
    # ai_msg = toolPicker.invoke(tool_query)
    # for tool_call in ai_msg.tool_calls:
    #     selected_tool = {"fetch_metrics": fetch_metrics,
    #                     "get_context": get_context
    #                     }[tool_call["name"].lower()]

    #     tool_msg = None 
    #     try:
    #         tool_msg = selected_tool.invoke(tool_call)
    #     except ValidationError as e:
    #         print(f"Error: {e.json()}")
    #     print(f"called {tool_call["name"].lower()} with {tool_call['args']}")
        # messages.append(f"connected {tool_call["name"].lower()} function returned {tool_msg.content}")
    # messages.append(f"connected {get_context(question=query, debug = verbose)} ")
    context = get_context(question=query, verbose = verbose)

    # if verbose:
    #     for message in messages:
    #         print(message)
    template = ChatPromptTemplate.from_template(f"""fullfill the query with the provided information
query      :{query}
information:{context}""")

    # RAG pipeline
    chain = ({"context": context , "question": RunnablePassthrough()}
        | template
        | chatbot_model
        | StrOutputParser())
    return chain.stream(query)
RAG("who manages this product")

who manages this product
<class 'langchain_pinecone.vectorstores.PineconeVectorStore'>
[{'collection': 'general_info', 'id': '67286e4f987ce18efd0c3ec3'}]
[{'collection': 'general_info', 'id': '67286e4f987ce18efd0c3ec3'}, {'collection': 'exclusions', 'id': '67286e50987ce18efd0c3ede'}]
[{'collection': 'general_info', 'id': '67286e4f987ce18efd0c3ec3'}, {'collection': 'exclusions', 'id': '67286e50987ce18efd0c3ede'}, {'collection': 'exclusions', 'id': '67286e50987ce18efd0c3edf'}]
[{'collection': 'general_info', 'id': '67286e4f987ce18efd0c3ec3'}, {'collection': 'exclusions', 'id': '67286e50987ce18efd0c3ede'}, {'collection': 'exclusions', 'id': '67286e50987ce18efd0c3edf'}, {'collection': 'requirements', 'id': '67286e50987ce18efd0c3ed1'}]
[{'collection': 'general_info', 'id': '67286e4f987ce18efd0c3ec3'}, {'collection': 'exclusions', 'id': '67286e50987ce18efd0c3ede'}, {'collection': 'exclusions', 'id': '67286e50987ce18efd0c3edf'}, {'collection': 'requirements', 'id': '67286e50987ce18efd0c3ed1'}

TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'generator'>

In [ ]:
#setup a simple intent classifier
def classify_intent(user_input:str) -> str:
    """classify the intent of the user input

current implementation uses a lightweight model (llama3.2:1B)
with few-shot prompting examples
for classifying 'normal', 'register', 'RAG' intents

"""
    return intent_classifier.invoke(f"""Classify the given input, use RAG if it is asking about insurance products,answer only with 'normal','register','RAG':

example:

Input: "Is there a contact number", Intent: RAG
Input: "How do I create a new account", Intent: register
Input: "How do I make a claim", Intent: RAG
Input: "Search the web for cat videos", Intent: normal
Input: "Help me register for this service", Intent: register
Input: "Where can I get started", Intent: register
Input: "What is your name", Intent: normal
Input: "What entities are attached to this service", Intent: RAG
Input: "What is your purpose", Intent: normal
Input: "Can I see some fund performance metrics", Intent: RAG
Input: "What is the weather in your country", Intent: normal
Input: "How can i register for an account", Intent: register
Input: "Who owns the product", Intent: RAG
Input: "Tell me how to subscribe", Intent: register
Input: "Guide me through the registration process", Intent: register
Input: "How do I sign up for the trial", Intent: register
Input: "Can you explain your features", Intent: normal
Input: "Sign me up", Intent: register
Input: "Can you assist me with enrolling", Intent: register
Input: "What services does the product provide", Intent: RAG
Input: "Where can i wash my dog", Intent: normal
Input: "Goodbye", Intent: normal
Input: "How do i pay for the service", Intent: RAG
Input: "how is my premium allocated", Intent: RAG
Input: "Hello", Intent: normal
Input: "What are the coverage options", Intent: RAG
Input: "What's the first step to register", Intent: register
Input: "What funds are involved", Intent: RAG
Input: "Where are you located", Intent: normal
Input: "Who do I contact for help", Intent: RAG
Input: "Can I pay with a credit card", Intent: RAG
Input: "Why should i trust you", Intent: normal
Input: "What should i get ready for enrollment", Intent: register
Input: "How are you", Intent": normal
Input: "What company distributes this service", Intent: RAG
Input: "how do i know you are not a scam", Intent: verify
Input: "Are there any additional charges", Intent: RAG
Input: "What can you tell me about the available insurance plans", Intent: RAG
Input: "how much do i need to pay for the insurance scheme", Intent: RAG
Input: "tell me about the available insurance plans", Intent: RAG
Input: "how did you get my number", Intent: verify
Input: "show me verification so i know this isn't a scam", Intent: verify
Input: "how do i know you are not scamming me", Intent: verify
Input: "Why should I sign up for this plan", Intent: RAG

Input: {user_input}""").content

classify_intent("who are you")

'normal'

In [28]:
def chat(user_input: str):
    intent = classify_intent(user_input)
    print(intent)
    if intent == "RAG":
        return get_context(user_input)
    elif intent == "register":
        print("register function work in progress")
    else:
        chain = chatbot_model | StrOutputParser()
        return chain.stream(user_input)
    


In [30]:
for chunk in chat("what is your name"):
    print(chunk, end="", flush=True)

normal
I don't have a personal name. I'm an AI designed to provide information and assist with tasks, but I don't have a personal identity or emotions. I exist solely to help users like you with their queries.

If you'd like, I can suggest some alternative names that might be helpful in our conversation. For example, I could be referred to as "Assistant" or "AI Companion." Let me know if there's anything else I can do to make our interaction more comfortable!